In [1]:
import pandas as pd
import datetime
import numpy as np
import os
import re
import operator

## Отчет Банки.ру

Отчет делается, на текущий момент, по трем файлам:

1) Файл из колл-центра (от Маши) "Акции НУ НИЧЕГО СЕБЕ", в нем заявки с сайта банки.ру на получение выгдоного курса. Необходимые столбцы Дата / время заявки и телефон

2) RFM анализ, выбирается подробный анализ за интересующий период (обычно неделя)

3) Отчет по ВОО за тот же период

Все файлы необходимо сохранить в одну папку. Задать эту папку в FOLDER.
Выбрать необходимый диапазон дат, задать в DATE_PERIOD.

Запустить все ячейки.
На выходе получим необходимые данные для заполнения в итоговый файл по отчету. Обычно, необходимый нам итоговый файл находится в X:\УМиР\2021\аналитика\Банки.Ру "Ну ничего себе на такую-то дату"

In [2]:
FOLDER = r'D:\FTP-DATA\Analytics\Banki_ru_nichosi\23052021'

DATE_PERIOD = '17.05.2021-23.05.2021'

In [3]:
os.chdir(FOLDER)

dates = list(map(lambda x: datetime.datetime.strptime(x, '%d.%m.%Y'), DATE_PERIOD.split('-')))

### Обработка дат и номеров телефонов из обзвона заявок banki.ru

In [4]:
nichosi = pd.read_excel('Акции НУ НИЧЕГО СЕБЕ.xlsx', usecols=['Дата / время заявки', 'телефон'], dtype=str)

nichosi = nichosi.dropna()

def to_date(x):
    
    try:
        x = x.replace(',', '.').replace(';', ':').replace('::', ':').replace('..', '.').replace('  ', ' ')
    except:
        print(x)
    x = x[:10].strip()
    
    if x.count('.') == 1:
        x = f'{x[:2]}.{x[2:]}'
        try:
            return pd.to_datetime(x, format='%d.%m.%Y')      
        except:
            return 'Проверить что за дата'
    elif all(['.' not in x[:3], '-' not in x[:3]]):
        try:
            return pd.to_datetime(x, format='%Y-%m-%d')
        except:
            return 'Проверить что за дата'        
    else:
        try:
            return pd.to_datetime(x, format='%d.%m.%Y')
        except:
            try:
                return pd.to_datetime(x, format='%d-%m-%Y')
            except:
                return 'Проверить что за дата'
            
nichosi['Дата'] = nichosi['Дата / время заявки'].apply(to_date)
            
not_a_date_indexes = nichosi[nichosi['Дата'] == 'Проверить что за дата'].index  



for index in not_a_date_indexes:
    nichosi['Дата'][index] = nichosi['Дата'][index - 1]
    
nichosi = nichosi[['Дата', 'телефон']]
    
nichosi = nichosi.sort_values(by='Дата', ignore_index=True)    

nichosi = nichosi[nichosi['Дата'].isin(pd.date_range(*(dates)))].reset_index(drop=True)    

promo_count = nichosi.shape[0] # кол-во промокодов из обзвона


def right_phone(x):
    x = x.replace(' ', '')
    pattern = r'[^\d]'
    x = re.sub(pattern, '', x)
    return f'{"7" * (11 - len(x))}{x}'

nichosi['телефон'] = nichosi['телефон'].apply(right_phone)






### считываем rfm и отчет по ВОО за нужный период, объединяем с данными из banki.ru

In [5]:
rfm = pd.read_excel([file for file in os.listdir() if all(['rfm' in file.lower(), '$' not in file])][0], usecols=['Телефон', 'Сумма операций за период, (рубл. экв.)'], dtype={'Телефон': str}, skiprows=2, sheet_name='RFM подробный')

voo = pd.read_excel([file for file in os.listdir() if all(['buyerrep' in file.lower(), '$' not in file])][0], usecols=['Телефон', 'Кол-во операций до начала периода'], dtype={'Телефон': str}, skiprows=2)

nichosi = nichosi.merge(rfm, how='left', left_on='телефон', right_on='Телефон').merge(voo, how='left', left_on='телефон', right_on='Телефон')

nichosi = nichosi[['Дата', 'телефон', 'Сумма операций за период, (рубл. экв.)', 'Кол-во операций до начала периода']]

### считаем необходимые величины для отчета

In [6]:
nichosi = nichosi.drop_duplicates(subset='телефон') # удаляем повторные номера телефонов чтобы два раза оборот не считать



indicators = {
    'кол-во сгенерированных промокодов': promo_count,
    'кол-во клиентов, совершивших операцию (общее)': nichosi['Сумма операций за период, (рубл. экв.)'].count(),
    'конверсия': f"{round((nichosi['Сумма операций за период, (рубл. экв.)'].count() / promo_count) * 100)}%",
    'кол-во клиентов по акции (предоставили документы)': (nichosi['Сумма операций за период, (рубл. экв.)'] >= 3100000).astype(int).sum(),
    'общая сумма операций клиентов с документами, руб': round(nichosi[nichosi['Сумма операций за период, (рубл. экв.)'] >= 3100000]['Сумма операций за период, (рубл. экв.)'].sum()),
    'кол-во клиентов без предоставления документов': (nichosi['Сумма операций за период, (рубл. экв.)'] < 3100000).astype(int).sum(),
    'общая сумма операций клиентов без документов, руб': round(nichosi[nichosi['Сумма операций за период, (рубл. экв.)'] < 3100000]['Сумма операций за период, (рубл. экв.)'].sum()),
    'БЫЛ': nichosi[(~nichosi['Сумма операций за период, (рубл. экв.)'].isnull()) & (nichosi['Кол-во операций до начала периода'] > 0)].shape[0],
    'НЕ БЫЛ': nichosi[(~nichosi['Сумма операций за период, (рубл. экв.)'].isnull()) & (nichosi['Кол-во операций до начала периода'] == 0)].shape[0]
    
}

In [7]:
pd.Series(indicators).to_excel(f'Показатели_{dates[0].date()}-{dates[1].date()}.xlsx')